In [1]:
import pandas as pd
from pathlib import Path

# Definimos rutas relativas al repo
DATA = Path("../data")  # subimos un nivel desde notebooks/
PROCESSED = DATA / "processed"
CLEAN = DATA / "clean"
PHI_DIR = CLEAN / "phishing"

# Crear carpeta si no existe
PHI_DIR.mkdir(parents=True, exist_ok=True)

# Paths de entrada y salida
CSV_SELECCION = PROCESSED / "eleccion_lgt_prototipo.csv"
CSV_PHISHING = PHI_DIR / "phishing_prototipo.csv"


In [3]:
# Path directo al phishing ya limpio
CSV_PHISHING = PHI_DIR / "phishing_prototipo.csv"

# Cargar
df_phishing = pd.read_csv(CSV_PHISHING)

print("Shape:", df_phishing.shape)
print("Columnas:", df_phishing.columns.tolist())
display(df_phishing.head(5))

# Nulos
print("\nNulos por columna:")
print(df_phishing.isna().sum())


Shape: (100, 17)
Columnas: ['url', 'matched_target', 'fuente', 'categoria', 'url_base', 'campaign_group', 'DS final', 'Motivos', '_in', 'domain', '_es', '_shortener', '_freehost', 'ruido_num', '_geo', '_es_kw', '_badness']


,url,matched_target,fuente,categoria,url_base,campaign_group,DS final,Motivos,_in,domain,_es,_shortener,_freehost,ruido_num,_geo,_es_kw,_badness
0,http://caixacapitalrisc.send2sign.es/login,caixabank,phishtank,banca,caixacapitalrisc.send2sign.es/login,send2sign.es/login,1.0,.es + Caixa en subdominio (SaaS tercero) + /lo...,1.0,caixacapitalrisc.send2sign.es,False,False,False,NaN,NaN,True,NaN
1,https://b9xja.dgnsvwrk.es/DwD9I@prmk4JYY2V/*to...,microsoft,phishtank,saas,b9xja.dgnsvwrk.es/dwd9i@prmk4jyy2v/*toto@micro...,dgnsvwrk.es/dwd9i@prmk4jyy2v,1.0,.es + marca (microsoft) en la ruta + ofuscació...,1.0,b9xja.dgnsvwrk.es,False,False,False,NaN,NaN,False,NaN
2,https://robllox.com.es,roblox,tweetfeed,gaming,robllox.com.es,robllox.com.es,1.0,typosquatting de Roblox + TLD .com.es (orienta...,1.0,robllox.com.es,True,False,False,NaN,NaN,False,NaN
3,https://zooominvitenotice.es/project/Windows/i...,zoom,tweetfeed,saas,zooominvitenotice.es/project/windows/invite.php,zooominvitenotice.es/project,1.0,typosquatting de Zoom + TLD .es + invite.php (...,1.0,zooominvitenotice.es,False,False,False,NaN,NaN,False,NaN
4,https://robiox.com.es,roblox,tweetfeed,gaming,robiox.com.es,robiox.com.es,1.0,typosquatting de Roblox + .com.es (orientación...,1.0,robiox.com.es,True,False,False,NaN,NaN,False,NaN



Nulos por columna:
url                0
matched_target     0
fuente             0
categoria          0
url_base           0
campaign_group     0
DS final           0
Motivos            1
_in                8
domain             8
_es                0
_shortener         0
_freehost          0
ruido_num         92
_geo              98
_es_kw             0
_badness          92
dtype: int64


In [5]:
# Reducir phishing a las columnas candidatas finales (sin guardar todavía)
rename_map = {
    "Motivos": "motivos",
    "campaign_group": "campaign"
}
df_phi_final = (
    df_phishing
    .rename(columns=rename_map)
    .loc[:, ["url", "categoria", "matched_target", "campaign", "motivos"]]
    .copy()
)

# Añadir columna label = 1
df_phi_final["label"] = 1

# Reordenar columnas
df_phi_final = df_phi_final[["url", "label", "categoria", "matched_target", "campaign", "motivos"]]

# Revisar shape y primeras filas
print("Shape phishing reducido:", df_phi_final.shape)
display(df_phi_final.head(10))

# Checks de distribución (sin guardar aún)
print("\nSectores:\n", df_phi_final["categoria"].value_counts())
print("\nTop targets:\n", df_phi_final["matched_target"].value_counts().head(10))


Shape phishing reducido: (100, 6)


,url,label,categoria,matched_target,campaign,motivos
0,http://caixacapitalrisc.send2sign.es/login,1,banca,caixabank,send2sign.es/login,.es + Caixa en subdominio (SaaS tercero) + /lo...
1,https://b9xja.dgnsvwrk.es/DwD9I@prmk4JYY2V/*to...,1,saas,microsoft,dgnsvwrk.es/dwd9i@prmk4jyy2v,.es + marca (microsoft) en la ruta + ofuscació...
2,https://robllox.com.es,1,gaming,roblox,robllox.com.es,typosquatting de Roblox + TLD .com.es (orienta...
3,https://zooominvitenotice.es/project/Windows/i...,1,saas,zoom,zooominvitenotice.es/project,typosquatting de Zoom + TLD .es + invite.php (...
4,https://robiox.com.es,1,gaming,roblox,robiox.com.es,typosquatting de Roblox + .com.es (orientación...
5,http://unicismadrid.es/wp-content/com/index/ch...,1,público,universidad de madrid,unicismadrid.es/wp-content,web .es comprometida (WordPress) + ruta /login...
6,https://authline-checkappr0v.com.es/7aIT03j82s...,1,genérico,genérico,authline-checkappr0v.com.es/7ait03j82stf28&sp_...,.com.es + semántica auth/approve con “0” (obfu...
7,https://koinbay-login.es,1,cripto,Coinbase,koinbay-login.es,typosquatting de Coinbase + .es
8,https://boxperience.es/cuenta/es-ing/ing,1,banca,ing,boxperience.es/cuenta,.es + posible web comprometida + marca ING en ...
9,http://authline-checkappr0v.com.es,1,genérico,genérico,authline-checkappr0v.com.es,.com.es + lexemas auth/approve con “0” → fake ...



Sectores:
 categoria
banca                 37
genérico              16
saas                   9
público                6
telecomunicaciones     6
cripto                 5
logística              5
streaming              5
gaming                 3
banca/teleco           2
logistica              2
publico                1
telecomunicaicones     1
energía                1
rrss                   1
Name: count, dtype: int64

Top targets:
 matched_target
genérico      33
ing           12
dgt            6
bbva           5
netflix        5
correos        3
santander      3
orange         3
Caja Rural     2
ionos          2
Name: count, dtype: int64


In [6]:
import unicodedata
import re

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

# 1) Definimos el conjunto canónico y el mapeo de correcciones
CATS_CANON = {
    "banca","telecomunicaciones","público","logística",
    "saas","streaming","gaming","cripto","energía","rrss","genérico"
}

MAP_CATS = {
    # variantes / typos → canónico
    "publico": "público",
    "logistica": "logística",
    "telecomunicaicones": "telecomunicaciones",
    "telecomunicaciones ": "telecomunicaciones",
    "banca/teleco": "mixto",   # ⚠️ marcamos ambiguas para revisión manual
    "logistica ": "logística",
    "energía ": "energía",
    "generico": "genérico",
    "genérico ": "genérico",
}

# 2) Normalización suave: trim, lower, y aplicar MAP_CATS
def norm_cat(x: str) -> str:
    raw = (x or "").strip()
    base = raw.lower()
    base_noacc = strip_accents(base)
    # primero corregimos typos conocidos
    base_corr = MAP_CATS.get(base, MAP_CATS.get(base_noacc, base))
    return base_corr

df_phi_norm = df_phi_final.copy()
df_phi_norm["categoria"] = df_phi_norm["categoria"].map(norm_cat)

# 3) Detectar valores fuera del set canónico
valores_categoria = df_phi_norm["categoria"].value_counts(dropna=False)
fuera_de_canon = sorted(set(df_phi_norm["categoria"]) - (CATS_CANON | {"mixto"}))  # permitimos "mixto" como bandera temporal

print("Distribución de categorías (tras normalizar):\n", valores_categoria, "\n")
print("Fuera del conjunto canónico (revisar a mano):", fuera_de_canon)
print("\nFilas 'mixto' (banca/teleco antes):")
display(df_phi_norm[df_phi_norm["categoria"]=="mixto"][["url","matched_target","categoria"]].head(10))


Distribución de categorías (tras normalizar):
 categoria
banca                 37
genérico              16
saas                   9
público                7
logística              7
telecomunicaciones     7
cripto                 5
streaming              5
gaming                 3
mixto                  2
energía                1
rrss                   1
Name: count, dtype: int64 

Fuera del conjunto canónico (revisar a mano): []

Filas 'mixto' (banca/teleco antes):


,url,matched_target,categoria
33,https://alerta-cliente.app,genérico,mixto
34,https://alertascliente.webcindario.com/,genérico,mixto


In [7]:
# Copiamos para no tocar el original
df_phi_fix = df_phi_final.copy()

# Reemplazar banca/teleco por genérico
mask = df_phi_fix["categoria"].str.strip().str.lower() == "banca/teleco".lower()
print("Filas afectadas:", mask.sum())

df_phi_fix.loc[mask, "categoria"] = "genérico"

# Comprobación
print("\nDistribución tras el cambio:\n", df_phi_fix["categoria"].value_counts())


Filas afectadas: 2

Distribución tras el cambio:
 categoria
banca                 37
genérico              18
saas                   9
público                6
telecomunicaciones     6
cripto                 5
logística              5
streaming              5
gaming                 3
logistica              2
publico                1
telecomunicaicones     1
energía                1
rrss                   1
Name: count, dtype: int64


In [8]:
# Copia de trabajo
df_phi_norm = df_phi_fix.copy()

# Mapeo de normalización de typos
MAP_CATS = {
    "logistica": "logística",
    "publico": "público",
    "telecomunicaicones": "telecomunicaciones"
}

# Aplicar reemplazos directos
df_phi_norm["categoria"] = df_phi_norm["categoria"].replace(MAP_CATS)

# Comprobación
print("Distribución tras normalización:\n")
print(df_phi_norm["categoria"].value_counts())


Distribución tras normalización:

categoria
banca                 37
genérico              18
saas                   9
público                7
logística              7
telecomunicaciones     7
cripto                 5
streaming              5
gaming                 3
energía                1
rrss                   1
Name: count, dtype: int64


In [9]:
# Set canónico esperado
CATS_CANON = {
    "banca","genérico","saas","público","logística",
    "telecomunicaciones","cripto","streaming","gaming",
    "energía","rrss"
}

# Verificar que no queda nada fuera del set esperado
fuera_canon = sorted(set(df_phi_norm["categoria"]) - CATS_CANON)

print("Valores fuera del set canónico:", fuera_canon)


Valores fuera del set canónico: []


In [10]:
df_phi_targets = df_phi_norm.copy()

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def norm_brand_min(b: str) -> str:
    b = (b or "").strip()
    # normalizar espacios y minúsculas
    b = re.sub(r"\s+", " ", b).lower()
    # quitar tildes para comparar
    b_noacc = strip_accents(b)
    # correcciones mínimas
    if b_noacc in {"generico", "generico "}:
        return "genérico"
    # devolver la forma sin tildes salvo el caso especial anterior
    # (si quieres preservar acentos en marcas como "caja rural", aquí no hace falta)
    return b_noacc

df_phi_targets["matched_target"] = df_phi_targets["matched_target"].map(norm_brand_min)

print("Top targets (tras normalizar mínimamente):")
print(df_phi_targets["matched_target"].value_counts().head(20))

# Revisión rápida: ¿quedan valores vacíos?
vacios = (df_phi_targets["matched_target"] == "").sum()
print("\nValores vacíos en matched_target:", vacios)
if vacios:
    display(df_phi_targets[df_phi_targets["matched_target"] == ""][["url","categoria","motivos"]].head(10))

Top targets (tras normalizar mínimamente):
matched_target
genérico         33
ing              12
dgt               6
bbva              5
netflix           5
ionos             3
santander         3
orange            3
correos           3
caixabank         2
dhl               2
caja rural        2
roblox            2
coinbase          2
dropbox           1
movistar          1
banco galicia     1
ddp               1
instagam          1
generica          1
Name: count, dtype: int64

Valores vacíos en matched_target: 0


In [11]:
# Copiamos para no tocar df_phi_norm original
df_phi_targets_fix = df_phi_targets.copy()

# Correcciones puntuales
MAP_TARGETS = {
    "generica": "genérico",
    "instagam": "instagram",
    "ddp": "dpd"
}

df_phi_targets_fix["matched_target"] = df_phi_targets_fix["matched_target"].replace(MAP_TARGETS)

# Comprobación de top valores
print("Top targets tras corrección puntual:\n")
print(df_phi_targets_fix["matched_target"].value_counts().head(20))


Top targets tras corrección puntual:

matched_target
genérico         34
ing              12
dgt               6
bbva              5
netflix           5
ionos             3
santander         3
orange            3
correos           3
caixabank         2
dhl               2
caja rural        2
coinbase          2
roblox            2
dropbox           1
banco galicia     1
dpd               1
instagram         1
bp                1
habbo             1
Name: count, dtype: int64


In [12]:
# Distribución completa de targets
pd.DataFrame(df_phi_targets_fix["matched_target"].value_counts())


,count
matched_target,
genérico,34
ing,12
dgt,6
bbva,5
netflix,5
ionos,3
santander,3
orange,3
correos,3


In [13]:
# Definir columnas finales en el orden correcto
cols_final = ["url","label","categoria","matched_target","campaign","motivos"]

# Seleccionar solo esas columnas del dataframe limpio
df_phishing_final = df_phi_targets_fix[cols_final].copy()

# Path de salida
CSV_PHISHING_FINAL = PHI_DIR / "phishing_final.csv"

# Guardar
df_phishing_final.to_csv(CSV_PHISHING_FINAL, index=False, encoding="utf-8")

print("✅ Guardado phishing_final.csv en:", CSV_PHISHING_FINAL)
print("Shape final:", df_phishing_final.shape)
display(df_phishing_final.head(10))


✅ Guardado phishing_final.csv en: ../data/clean/phishing/phishing_final.csv
Shape final: (100, 6)


,url,label,categoria,matched_target,campaign,motivos
0,http://caixacapitalrisc.send2sign.es/login,1,banca,caixabank,send2sign.es/login,.es + Caixa en subdominio (SaaS tercero) + /lo...
1,https://b9xja.dgnsvwrk.es/DwD9I@prmk4JYY2V/*to...,1,saas,microsoft,dgnsvwrk.es/dwd9i@prmk4jyy2v,.es + marca (microsoft) en la ruta + ofuscació...
2,https://robllox.com.es,1,gaming,roblox,robllox.com.es,typosquatting de Roblox + TLD .com.es (orienta...
3,https://zooominvitenotice.es/project/Windows/i...,1,saas,zoom,zooominvitenotice.es/project,typosquatting de Zoom + TLD .es + invite.php (...
4,https://robiox.com.es,1,gaming,roblox,robiox.com.es,typosquatting de Roblox + .com.es (orientación...
5,http://unicismadrid.es/wp-content/com/index/ch...,1,público,universidad de madrid,unicismadrid.es/wp-content,web .es comprometida (WordPress) + ruta /login...
6,https://authline-checkappr0v.com.es/7aIT03j82s...,1,genérico,genérico,authline-checkappr0v.com.es/7ait03j82stf28&sp_...,.com.es + semántica auth/approve con “0” (obfu...
7,https://koinbay-login.es,1,cripto,coinbase,koinbay-login.es,typosquatting de Coinbase + .es
8,https://boxperience.es/cuenta/es-ing/ing,1,banca,ing,boxperience.es/cuenta,.es + posible web comprometida + marca ING en ...
9,http://authline-checkappr0v.com.es,1,genérico,genérico,authline-checkappr0v.com.es,.com.es + lexemas auth/approve con “0” → fake ...


In [14]:
# Path de las legítimas
LEG_DIR = CLEAN / "legitimas"
CSV_LEGITIMAS = LEG_DIR / "legítimas_prototipo.csv"

# Cargar CSV
df_legit_raw = pd.read_csv(CSV_LEGITIMAS)

print("Shape:", df_legit_raw.shape)
print("Columnas:", df_legit_raw.columns.tolist())

# Primeras filas
display(df_legit_raw.head(10))

# Valores nulos por columna
print("\nNulos por columna:")
print(df_legit_raw.isna().sum())


Shape: (100, 4)
Columnas: ['url', 'matched_target', 'categoria', 'legítima']


,url,matched_target,categoria,legítima
0,http://caixacapitalrisc.send2sign.es/login,caixabank,banca,https://www.caixabank.es/particular/banca-digi...
1,https://b9xja.dgnsvwrk.es/DwD9I@prmk4JYY2V/*to...,microsoft,saas,https://accounts.google.com/ServiceLogin?servi...
2,https://robllox.com.es,roblox,gaming,https://www.roblox.com/es/upgrades/robux?ctx=n...
3,https://zooominvitenotice.es/project/Windows/i...,zoom,saas,https://zoom.us/es/join
4,https://robiox.com.es,roblox,gaming,https://www.roblox.com/es/login
5,http://unicismadrid.es/wp-content/com/index/ch...,universidad de madrid,público,https://www.unicismadrid.es
6,https://authline-checkappr0v.com.es/7aIT03j82s...,genérico,genérico,https://www.pass.carrefour.es/zona-cliente/
7,https://koinbay-login.es,Coinbase,cripto,https://help.coinbase.com/es-es
8,https://boxperience.es/cuenta/es-ing/ing,ing,banca,https://ing.ingdirect.es/pfm/#login
9,http://authline-checkappr0v.com.es,genérico,genérico,https://www.financieraelcorteingles.es/es/pago...



Nulos por columna:
url               0
matched_target    0
categoria         0
legítima          0
dtype: int64


In [15]:
# Crear dataframe de legítimas a partir de la columna 'legítima'
df_legit_final = df_legit_raw[["legítima","categoria","matched_target"]].copy()

# Renombrar columna 'legítima' a 'url'
df_legit_final = df_legit_final.rename(columns={"legítima": "url"})

# Añadir columna 'label' = 0
df_legit_final["label"] = 0

# Añadir columna 'notas' (vacía de momento, para rellenar manualmente)
df_legit_final["notas"] = ""

# Reordenar columnas
df_legit_final = df_legit_final[["url","label","categoria","matched_target","notas"]]

print("Shape legitimas_final:", df_legit_final.shape)
display(df_legit_final.head(10))


Shape legitimas_final: (100, 5)


,url,label,categoria,matched_target,notas
0,https://www.caixabank.es/particular/banca-digi...,0,banca,caixabank,
1,https://accounts.google.com/ServiceLogin?servi...,0,saas,microsoft,
2,https://www.roblox.com/es/upgrades/robux?ctx=n...,0,gaming,roblox,
3,https://zoom.us/es/join,0,saas,zoom,
4,https://www.roblox.com/es/login,0,gaming,roblox,
5,https://www.unicismadrid.es,0,público,universidad de madrid,
6,https://www.pass.carrefour.es/zona-cliente/,0,genérico,genérico,
7,https://help.coinbase.com/es-es,0,cripto,Coinbase,
8,https://ing.ingdirect.es/pfm/#login,0,banca,ing,
9,https://www.financieraelcorteingles.es/es/pago...,0,genérico,genérico,


In [16]:
import re, unicodedata
import pandas as pd

df_legit_norm = df_legit_final.copy()

# ---- helpers
def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def norm_spaces_lower(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    return s.lower()

# ---- normalización mínima de categoria (mismos criterios que phishing)
MAP_CATS_LEGIT = {
    "publico": "público",
    "logistica": "logística",
    "telecomunicaicones": "telecomunicaciones",
    "banca/teleco": "genérico",
}
df_legit_norm["categoria"] = df_legit_norm["categoria"].replace(MAP_CATS_LEGIT)

# ---- normalización mínima de matched_target
def norm_brand_min(b: str) -> str:
    b = norm_spaces_lower(b)
    b_noacc = strip_accents(b)
    # correcciones puntuales conocidas
    if b_noacc in {"generico", "generica"}:
        return "genérico"
    return b_noacc

df_legit_norm["matched_target"] = df_legit_norm["matched_target"].map(norm_brand_min)

# ---- QC rápido
print("Distribución por categoria (legítimas, tras normalizar):\n",
      df_legit_norm["categoria"].value_counts(), "\n")

print("Top targets (legítimas, tras normalizar):\n",
      df_legit_norm["matched_target"].value_counts().head(20), "\n")

# ¿duplicados?
dups = df_legit_norm["url"].duplicated().sum()
print("Duplicados por URL (legítimas):", dups)

# Vista de muestra
display(df_legit_norm.head(10))


Distribución por categoria (legítimas, tras normalizar):
 categoria
banca                 37
genérico              18
saas                   9
público                7
logística              7
telecomunicaciones     7
cripto                 5
streaming              5
gaming                 3
energía                1
redes sociales         1
Name: count, dtype: int64 

Top targets (legítimas, tras normalizar):
 matched_target
genérico         34
ing              12
dgt               6
bbva              5
netflix           5
santander         4
ionos             3
orange            3
correos           3
dhl               2
caja rural        2
caixabank         2
coinbase          2
roblox            2
vodafone          1
instagam          1
bp                1
habbo             1
dpd               1
banco galicia     1
Name: count, dtype: int64 

Duplicados por URL (legítimas): 0


,url,label,categoria,matched_target,notas
0,https://www.caixabank.es/particular/banca-digi...,0,banca,caixabank,
1,https://accounts.google.com/ServiceLogin?servi...,0,saas,microsoft,
2,https://www.roblox.com/es/upgrades/robux?ctx=n...,0,gaming,roblox,
3,https://zoom.us/es/join,0,saas,zoom,
4,https://www.roblox.com/es/login,0,gaming,roblox,
5,https://www.unicismadrid.es,0,público,universidad de madrid,
6,https://www.pass.carrefour.es/zona-cliente/,0,genérico,genérico,
7,https://help.coinbase.com/es-es,0,cripto,coinbase,
8,https://ing.ingdirect.es/pfm/#login,0,banca,ing,
9,https://www.financieraelcorteingles.es/es/pago...,0,genérico,genérico,


In [17]:
# Selección final de columnas en orden
cols_legit_final = ["url","label","categoria","matched_target","notas"]

df_legit_final_out = df_legit_norm[cols_legit_final].copy()

# Path de salida
CSV_LEGIT_FINAL = LEG_DIR / "legitimas_final.csv"

# Guardar
df_legit_final_out.to_csv(CSV_LEGIT_FINAL, index=False, encoding="utf-8")

print("✅ Guardado:", CSV_LEGIT_FINAL, "→", df_legit_final_out.shape)
display(df_legit_final_out.head(10))


✅ Guardado: ../data/clean/legitimas/legitimas_final.csv → (100, 5)


,url,label,categoria,matched_target,notas
0,https://www.caixabank.es/particular/banca-digi...,0,banca,caixabank,
1,https://accounts.google.com/ServiceLogin?servi...,0,saas,microsoft,
2,https://www.roblox.com/es/upgrades/robux?ctx=n...,0,gaming,roblox,
3,https://zoom.us/es/join,0,saas,zoom,
4,https://www.roblox.com/es/login,0,gaming,roblox,
5,https://www.unicismadrid.es,0,público,universidad de madrid,
6,https://www.pass.carrefour.es/zona-cliente/,0,genérico,genérico,
7,https://help.coinbase.com/es-es,0,cripto,coinbase,
8,https://ing.ingdirect.es/pfm/#login,0,banca,ing,
9,https://www.financieraelcorteingles.es/es/pago...,0,genérico,genérico,
